# Multi-QueryRetriever
큰 장점. 유저가 생각보다 쿼리를 친절하게 작성을 안한다. 이것을 친절한 표현으로 바꿔주는 후처리적인 개념

우리가 쿼리를 가지고 문서검색을 할때, 날것의 질문보다, llm의 질문이 더 성능이 좋다고 기대할 수 있다.

In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH10-Retriever


In [35]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = Chroma.from_documents(documents, embeddings)
retriever = vector_store.as_retriever()

query = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = retriever.invoke(query)

len(relevant_docs)


4

In [36]:
print(relevant_docs)

[Document(metadata={'description': 'OpenAI의 새로운 Assistants API는 상태 유지 대화와 강력한 도구 접근성을 제공합니다. OpenAI Assistants API 튜토리얼: Code Interpreter, Retrieval, Functions 활용법에 대하여 알아보겠습니다.', 'language': 'ko', 'source': 'https://teddylee777.github.io/openai/openai-assistant-tutorial/', 'title': '[Assistants API] Code Interpreter, Retrieval, Functions 활용법 - 테디노트'}, page_content='34 분 소요\n        \n      \n\n\n\n\n\n 목차\nAssistants API 개요Assistants API 의 탄생 배경Assistants API의 기본 요소환경설정Python 라이브러리 설정Helper 함수API KEY 설정Assistants APIPlayground 에서 Assistants 생성Assistants API 로 생성Threads(스레드)Run(실행)Message(메시지)함수화전체코드(템플릿 코드)Assistant + tools(도구)도구1: Code Interpreter(코드 인터프리터)실행단계(steps)도구2: Retrieval(검색)파일 업로드Assistant 설정 업데이트(도구 추가)도구3: Functions(함수)required_action퀴즈 생성기 전체코드1) 환경설정2) 파일 업로드3) Function Schema 정의4) 퀴즈 요청5) 피드백끝으로Reference\n\n\nOpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Fu

In [37]:
# llm을 통해 멀티쿼리를 생성. vector_store를 넣어준다.

In [38]:
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = vector_store.as_retriever(),
    llm=llm
)

In [39]:
# 쿼리에 대한 로깅 설정
import logging

logging.basicConfig()
logging.getLogger("langchain_retrievers.multi_query").setLevel(logging.INFO)

In [40]:
# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
# 문서 검색
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

검색된 문서 개수: 3
34 분 소요
        
      





 목차
Assistants API 개요Assistants API 의 탄생 배경Assistants API의 기본 요소환경설정Python 라이브러리 설정Helper 함수API KEY 설정Assistants APIPlayground 에서 Assistants 생성Assistants API 로 생성Threads(스레드)Run(실행)Message(메시지)함수화전체코드(템플릿 코드)Assistant + tools(도구)도구1: Code Interpreter(코드 인터프리터)실행단계(steps)도구2: Retrieval(검색)파일 업로드Assistant 설정 업데이트(도구 추가)도구3: Functions(함수)required_action퀴즈 생성기 전체코드1) 환경설정2) 파일 업로드3) Function Schema 정의4) 퀴즈 요청5) 피드백끝으로Reference


OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


In [47]:
ddrelevant_docs[1].

{'description': 'OpenAI의 새로운 Assistants API는 상태 유지 대화와 강력한 도구 접근성을 제공합니다. OpenAI Assistants API 튜토리얼: Code Interpreter, Retrieval, Functions 활용법에 대하여 알아보겠습니다.',
 'language': 'ko',
 'source': 'https://teddylee777.github.io/openai/openai-assistant-tutorial/',
 'title': '[Assistants API] Code Interpreter, Retrieval, Functions 활용법 - 테디노트'}

# LECL Chain 활용하는 방법

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
You are an AI language model assistant. 
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. 
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION: 
{question}

#Answer in Korean:

"""

prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt| llm | StrOutputParser()
)
# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

# 체인을 실행하여 생성된 다중 쿼리를 확인합니다.
multi_queries = custom_multiquery_chain.invoke(question)
# 결과를 확인합니다.(5개 질문 생성)
multi_queries

'OpenAI Assistant API의 Functions 활용 방법에 대해 설명해 주세요.  \nOpenAI Assistant API에서 Functions를 사용하는 절차를 알려주세요.  \nOpenAI Assistant API의 Functions 기능에 대한 안내를 부탁드립니다.  \nOpenAI Assistant API의 Functions 사용에 대한 정보를 제공해 주세요.  \nOpenAI Assistant API에서 Functions를 어떻게 활용할 수 있는지 궁금합니다.  '

In [13]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=custom_multiquery_chain, retriever=vector_store.as_retriever()
)

In [14]:
# 결과
relevant_docs = multiquery_retriever.invoke(question)

In [15]:
relevant_docs

[Document(metadata={'description': 'OpenAI의 새로운 Assistants API는 상태 유지 대화와 강력한 도구 접근성을 제공합니다. OpenAI Assistants API 튜토리얼: Code Interpreter, Retrieval, Functions 활용법에 대하여 알아보겠습니다.', 'language': 'ko', 'source': 'https://teddylee777.github.io/openai/openai-assistant-tutorial/', 'title': '[Assistants API] Code Interpreter, Retrieval, Functions 활용법 - 테디노트'}, page_content='34 분 소요\n        \n      \n\n\n\n\n\n 목차\nAssistants API 개요Assistants API 의 탄생 배경Assistants API의 기본 요소환경설정Python 라이브러리 설정Helper 함수API KEY 설정Assistants APIPlayground 에서 Assistants 생성Assistants API 로 생성Threads(스레드)Run(실행)Message(메시지)함수화전체코드(템플릿 코드)Assistant + tools(도구)도구1: Code Interpreter(코드 인터프리터)실행단계(steps)도구2: Retrieval(검색)파일 업로드Assistant 설정 업데이트(도구 추가)도구3: Functions(함수)required_action퀴즈 생성기 전체코드1) 환경설정2) 파일 업로드3) Function Schema 정의4) 퀴즈 요청5) 피드백끝으로Reference\n\n\nOpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Fu

In [16]:
# 결과
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

검색된 문서 개수: 5
34 분 소요
        
      





 목차
Assistants API 개요Assistants API 의 탄생 배경Assistants API의 기본 요소환경설정Python 라이브러리 설정Helper 함수API KEY 설정Assistants APIPlayground 에서 Assistants 생성Assistants API 로 생성Threads(스레드)Run(실행)Message(메시지)함수화전체코드(템플릿 코드)Assistant + tools(도구)도구1: Code Interpreter(코드 인터프리터)실행단계(steps)도구2: Retrieval(검색)파일 업로드Assistant 설정 업데이트(도구 추가)도구3: Functions(함수)required_action퀴즈 생성기 전체코드1) 환경설정2) 파일 업로드3) Function Schema 정의4) 퀴즈 요청5) 피드백끝으로Reference


OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용
